In [1]:
import os, sys
import glob
import tqdm
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

/Users/admin/Documents/Github/SkinCancer/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Data exploration

In [3]:
sys.path.insert(0,'/Users/admin/Documents/GitHub/SkinCancer/src')
from dataset import SkinCancer

/Users/admin/Documents/Github/SkinCancer/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
df = pd.read_csv('../csv/new_meta.csv')
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,../data/HAM10k/HAM10000_images/ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,../data/HAM10k/HAM10000_images/ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,../data/HAM10k/HAM10000_images/ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,../data/HAM10k/HAM10000_images/ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,../data/HAM10k/HAM10000_images/ISIC_0031633.jpg


In [16]:

classes_to_augment = ['akiec', 'bcc']
# df[df['dx'].isin(['nv'])].shape
img_path = df.iloc[:,-1]
img_path[df['dx'].isin(classes_to_augment)]


2462     ../data/HAM10k/HAM10000_images/ISIC_0028155.jpg
2463     ../data/HAM10k/HAM10000_images/ISIC_0034093.jpg
2464     ../data/HAM10k/HAM10000_images/ISIC_0029230.jpg
2465     ../data/HAM10k/HAM10000_images/ISIC_0031513.jpg
2466     ../data/HAM10k/HAM10000_images/ISIC_0032384.jpg
                              ...                       
10009    ../data/HAM10k/HAM10000_images/ISIC_0031430.jpg
10010    ../data/HAM10k/HAM10000_images/ISIC_0033084.jpg
10011    ../data/HAM10k/HAM10000_images/ISIC_0033550.jpg
10012    ../data/HAM10k/HAM10000_images/ISIC_0033536.jpg
10013    ../data/HAM10k/HAM10000_images/ISIC_0032854.jpg
Name: image_path, Length: 841, dtype: object

In [17]:
counts = df['dx'].value_counts().to_dict()

In [18]:
counts

{'nv': 6705,
 'mel': 1113,
 'bkl': 1099,
 'bcc': 514,
 'akiec': 327,
 'vasc': 142,
 'df': 115}

In [ ]:
[1 - counts[i]/df.shape[0] for i in self.classes]

In [ ]:
df['image_path'][0]

In [4]:
root_path = '../data/HAM10k/HAM10000_images'


In [5]:
ds = SkinCancer(root_path,'../data/train.csv')

In [ ]:
ds.classes_all

In [23]:
import torch
from sklearn.utils import class_weight

In [ ]:
y = df['dx'].to_numpy()

In [ ]:
del class_weights

In [24]:
class_weights=class_weight.compute_class_weight('balanced',classes=np.unique(ds.classes),y=np.array(ds.classes_all))
class_weights=torch.tensor(class_weights,dtype=torch.float)

In [ ]:
class_weights.shape

In [25]:
class_weights

tensor([ 4.3686,  2.7848,  1.3021, 12.4410,  1.2860,  0.2134, 10.0401])

In [26]:
ds.classes

['nv', 'bkl', 'mel', 'bcc', 'akiec', 'df', 'vasc']

In [14]:
df_minority = df[df['dx']!='nv']

In [16]:
df_minority['dx'].value_counts()

mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [30]:
dftrain = pd.read_csv('../data/augment2_train.csv')
dftest = pd.read_csv('../data/augment2_test.csv')

In [31]:
dftrain

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0004756,ISIC_0030442_1,df,histo,55.0,male,lower extremity,../data/Augmented_rename/ISIC_0030442_1.jpg
1,HAM_0005989,ISIC_0033123_41,vasc,histo,25.0,female,back,../data/Augmented_rename/ISIC_0033123_41.jpg
2,HAM_0007225,ISIC_0031429_34,df,histo,75.0,male,lower extremity,../data/Augmented_rename/ISIC_0031429_34.jpg
3,HAM_0007410,ISIC_0031457_47,df,histo,65.0,male,foot,../data/Augmented_rename/ISIC_0031457_47.jpg
4,HAM_0002639,ISIC_0025249_24,vasc,consensus,30.0,female,lower extremity,../data/Augmented_rename/ISIC_0025249_24.jpg
...,...,...,...,...,...,...,...,...
34898,HAM_0004984,ISIC_0032879_1,mel,histo,80.0,male,face,../data/Augmented_rename/ISIC_0032879_1.jpg
34899,HAM_0001852,ISIC_0033749_0,vasc,histo,80.0,male,lower extremity,../data/Augmented_rename/ISIC_0033749_0.jpg
34900,HAM_0001616,ISIC_0029669_7,bcc,histo,65.0,female,back,../data/Augmented_rename/ISIC_0029669_7.jpg
34901,HAM_0000940,ISIC_0033608_36,vasc,histo,35.0,female,lower extremity,../data/Augmented_rename/ISIC_0033608_36.jpg


In [33]:
dftest[dftest['dx']=='nv']

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
12,HAM_0003767,ISIC_0024604,nv,histo,60.0,male,upper extremity,../data/HAM10k/HAM10000_images/ISIC_0024604.jpg
18,HAM_0000366,ISIC_0027147,nv,histo,75.0,male,back,../data/HAM10k/HAM10000_images/ISIC_0027147.jpg
20,HAM_0001112,ISIC_0024359,nv,follow_up,40.0,male,back,../data/HAM10k/HAM10000_images/ISIC_0024359.jpg
24,HAM_0003836,ISIC_0031802,nv,histo,55.0,male,unknown,../data/HAM10k/HAM10000_images/ISIC_0031802.jpg
26,HAM_0001398,ISIC_0027934,nv,follow_up,65.0,female,back,../data/HAM10k/HAM10000_images/ISIC_0027934.jpg
...,...,...,...,...,...,...,...,...
8691,HAM_0004066,ISIC_0026390,nv,follow_up,30.0,female,back,../data/HAM10k/HAM10000_images/ISIC_0026390.jpg
8693,HAM_0005143,ISIC_0030303,nv,histo,40.0,male,chest,../data/HAM10k/HAM10000_images/ISIC_0030303.jpg
8695,HAM_0007295,ISIC_0031380,nv,histo,60.0,female,chest,../data/HAM10k/HAM10000_images/ISIC_0031380.jpg
8702,HAM_0004149,ISIC_0033880,nv,histo,45.0,female,foot,../data/HAM10k/HAM10000_images/ISIC_0033880.jpg


In [47]:
df = pd.read_csv('../data/new_augmented.csv')
df['dx'].value_counts().sum()

46996

In [46]:
len(glob.glob('../data/Augmented_rename/*.jpg'))

37131

In [34]:
import matplotlib.pyplot as plt



In [42]:
dftrain['dx'].value_counts().keys().to_list()

['nv', 'df', 'vasc', 'akiec', 'bcc', 'bkl', 'mel']

In [48]:
dftrain.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0004756,ISIC_0030442_1,df,histo,55.0,male,lower extremity,../data/Augmented_rename/ISIC_0030442_1.jpg
1,HAM_0005989,ISIC_0033123_41,vasc,histo,25.0,female,back,../data/Augmented_rename/ISIC_0033123_41.jpg
2,HAM_0007225,ISIC_0031429_34,df,histo,75.0,male,lower extremity,../data/Augmented_rename/ISIC_0031429_34.jpg
3,HAM_0007410,ISIC_0031457_47,df,histo,65.0,male,foot,../data/Augmented_rename/ISIC_0031457_47.jpg
4,HAM_0002639,ISIC_0025249_24,vasc,consensus,30.0,female,lower extremity,../data/Augmented_rename/ISIC_0025249_24.jpg


In [89]:

def plot_samples(df):
    classes = df['dx'].value_counts().keys().to_list()
    r = c = len(classes)
    plt.figure(figsize=(10,10)) # specifying the overall grid size
    
    for cl in classes:
        for i in range(r*c):
        
            
            img_path = df[df['dx']==cl].iloc[i,-1]
            # img_path = [df.iloc[i,-1] if df.iloc[i,2] == str(c)]
            # print(c, img_path)
            plt.subplot(r,c,i+1)    # the number of images in the grid is 5*5 (25)
        # img_path = df.iloc[i,-1] for 
        
            plt.imshow(cv2.imread(img_path))
            plt.title(str(cl))


            plt.show()

In [ ]:
plot_samples(dftrain)